Notes

* some block numbers have XX used as anonymization; eg. 030XX means somewhere on 3000 block, 014XX means somwhere on 1400 block
* will either need to use the street name or approximate address (030XX -> 3000/3012/3014)
* some address are truncated
* "&" used for intersections

* Some coordinates do not exactly match up to given address; many are off by an order of thousands of feet
* The final df in this case will have a lat,lng, final_lat, final_lng columns; final_lat, final_lng is either:
    *** the coords specified in the original df
    ***the imputed coords based on address estimation

In [2]:
import numpy as np
import pandas as pd
import re
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter



In [3]:
# read in SOPP data
df = pd.read_csv("policing.csv")
df.head()

/var/folders/b7/4yf7b9tn1t1ck0mv8h93dssc0000gn/T/ipykernel_2223/1773643332.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("policing.csv")


,raw_row_number,date,time,location,lat,lng,district,zone,subject_age,subject_race,...,search_person,search_vehicle,search_basis,reason_for_stop,vehicle_color,vehicle_make,vehicle_model,vehicle_year,raw_actions_taken,raw_subject_race
0,1,2010-01-01,4260,NaN,NaN,NaN,6,E,26.0,black,...,False,False,NaN,TRAFFIC VIOLATION,BLACK,DODGE,CARAVAN,2005.0,NaN,BLACK
1,9087,2010-01-01,5340,NaN,NaN,NaN,7,C,37.0,black,...,False,False,NaN,TRAFFIC VIOLATION,BLUE,NISSAN,MURANO,2005.0,NaN,BLACK
2,9086,2010-01-01,5340,NaN,NaN,NaN,7,C,37.0,black,...,False,False,NaN,TRAFFIC VIOLATION,BLUE,NISSAN,MURANO,2005.0,NaN,BLACK
3,267,2010-01-01,50400,NaN,NaN,NaN,7,I,96.0,black,...,False,False,NaN,TRAFFIC VIOLATION,GRAY,JEEP,GRAND CHEROKEE,2003.0,NaN,BLACK
4,2,2010-01-01,7560,NaN,NaN,NaN,5,D,17.0,black,...,False,False,NaN,CALL FOR SERVICE,NaN,NaN,NaN,NaN,NaN,BLACK


In [4]:
df["reason_for_stop"].unique()

array(['TRAFFIC VIOLATION', 'CALL FOR SERVICE', 'SUSPECT PERSON',
       'CITIZEN CONTACT', 'OTHER', 'FLAGGED DOWN', 'JUVENILE VIOLATION',
       'SUSPECT VEHICLE', 'CRIMINAL VIOLATION', 'PRESENT AT CRIME SCENE',
       'TRAFFIC VIOLATION|OTHER',
       'SUSPECT VEHICLE|SUSPECT PERSON|SUSPECT PERSON',
       'TRAFFIC VIOLATION|CALL FOR SERVICE',
       'CITIZEN CONTACT|FLAGGED DOWN',
       'CALL FOR SERVICE|TRAFFIC VIOLATION',
       'CRIMINAL VIOLATION|TRAFFIC VIOLATION',
       'TRAFFIC VIOLATION|OTHER|OTHER'], dtype=object)

In [ ]:
# location data sample
print(df['location'].dropna().sample(30, random_state=42).tolist())


#proportion of data that has a location
print(f"proportion of data that has location: {(df['location'].notna().mean()) * 100}")

# unique locations
unique_locations = df[df['lat'].isna() & df['location'].notna()]['location'].unique()
print(f"unique locations: {len(unique_locations)}")




['030XX Elysian Fields Ave', '014XX N Claiborne Ave', '030XX Tchoupitoulas St', 'Leonidas St & S Claiborne Ave', 'Decatur St & Toulouse St', '017XX Spain St', 'Bloomingdale Ct & Erato St', '007XX Canal St', 'Memphis St & Robert E Lee Blvd', 'Forshey St & S Carrollton Ave', 'Palmyra St & S Broad St', 'N Claiborne Ave & Saint Maurice Ave', '010XX Caffin Ave', 'Canal Blvd & Filmore Ave', 'Elysian Fields Ave & Robert E Le', 'Martin Luther King Jr Blvd & S Liberty St', '001XX 40th St', 'Igor Sikorsky Dr & Stars & Str', '004XX Belleville St', 'New Orleans St & N Rocheblave St', '021XX Franklin Ave', '012XX Royal St', 'S Carrollton Ave & Tulane Ave', '032XX Garden Oaks Dr', 'Tchoupitoulas St & Jefferson Ave', '018XX Saint Thomas St', '014XX Saint Charles Ave', '018XX Marais St', 'Saint Andrew St & Magnolia St', 'Gen Diaz St & Porteous St']
proportion of data that has location: 81.25610241909656
unique locations: 12995


In [5]:
# proportion of data with coordinates
print("proportion of data that has coordinates")
print(100 - (sum(df["lat"].isnull())/len(df["lat"]) * 100))
print(100 - (sum(df["lng"].isnull())/len(df["lng"]) * 100))

proportion of data that has coordinates
50.85180006717543
50.85180006717543


In [ ]:
# get rows missing lat/lng but having location 
missing_coords = df[df["lat"].isna() & df["location"].notna()]

# get unique locations
unique_locs_df = pd.DataFrame(missing_coords['location'].unique(), columns=['location'])

print(f"Total rows missing coords: {len(missing_coords)}")
print(f"Unique locations to geocode: {len(unique_locs_df)}")


Total rows missing coords: 155698
Unique locations to geocode: 12995


In [ ]:
# string cleaning 
def clean_location(loc):
    if not isinstance(loc, str):
        return None
    loc = loc.strip()
    loc = re.sub(r'(\d+)XX', lambda m: str(int(m.group(1))) + '00', loc)
    return loc + ", New Orleans, LA"

unique_locs_df['query'] = unique_locs_df['location'].apply(clean_location)
unique_locs_df = unique_locs_df.drop(columns=['imp_lat', 'imp_lng'], errors='ignore')
unique_locs_df.head()

,location,query
0,029XX Vespasian Blvd,"2900 Vespasian Blvd, New Orleans, LA"
1,033XX Gen De Gaulle Dr,"3300 Gen De Gaulle Dr, New Orleans, LA"
2,076XX Fieldston Rd,"7600 Fieldston Rd, New Orleans, LA"
3,005XX Bourbon St,"500 Bourbon St, New Orleans, LA"
4,020XX La Salle St,"2000 La Salle St, New Orleans, LA"


In [ ]:
# this cell will download the csv locally; takes 1+ hours

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm
import os

geolocator = Nominatim(user_agent="sds357-project")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, error_wait_seconds=5)

# Resume from where left off if file exists
if os.path.exists('geocoded_locations.csv'):
    already_done = pd.read_csv('geocoded_locations.csv')
    unique_locs_df = unique_locs_df.merge(already_done[['location', 'imp_lat', 'imp_lng']], on='location', how='left')
    to_geocode = unique_locs_df[unique_locs_df['imp_lat'].isna()]
    print(f"Resuming — {len(already_done) - to_geocode.shape[0]} already done, {len(to_geocode)} remaining")
else:
    unique_locs_df['imp_lat'] = None
    unique_locs_df['imp_lng'] = None
    to_geocode = unique_locs_df
    print(f"Starting fresh — {len(to_geocode)} locations to geocode")

for idx, row in tqdm(to_geocode.iterrows(), total=len(to_geocode)):
    result = geocode(row['query'])
    unique_locs_df.at[idx, 'imp_lat'] = result.latitude if result else None
    unique_locs_df.at[idx, 'imp_lng'] = result.longitude if result else None
    if idx % 100 == 0:
        unique_locs_df[['location', 'query', 'imp_lat', 'imp_lng']].to_csv('geocoded_locations.csv', index=False)

# Final save
unique_locs_df[['location', 'query', 'imp_lat', 'imp_lng']].to_csv('geocoded_locations.csv', index=False)
print(f"Successfully geocoded: {unique_locs_df['imp_lat'].notna().sum()} / {len(unique_locs_df)}")

Resuming — 12295 already done, 700 remaining


  0%|          | 0/700 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [17]:
geocoded_df = pd.read_csv('geocoded_locations.csv')

print(f"total locations: {len(geocoded_df)}")
print(f"successfully geocoded: {geocoded_df['imp_lat'].notna().sum()}")
print(f"success rate: {geocoded_df['imp_lat'].notna().mean()}")

total locations: 12995
successfully geocoded: 12295
success rate: 0.9461331281262024


In [ ]:
# bounding box filter
lat_min, lat_max = 29.866661, 30.19866
lng_min, lng_max = -90.140074, -89.625053

geocoded_df['valid'] = (
    geocoded_df['imp_lat'].between(lat_min, lat_max) &
    geocoded_df['imp_lng'].between(lng_min, lng_max)
)

# set out of bounds coords to None
geocoded_df.loc[~geocoded_df['valid'], ['imp_lat', 'imp_lng']] = None

print(f"valid: {geocoded_df['imp_lat'].notna().sum()}")
print(f"out of bounds: {(~geocoded_df['valid'] & geocoded_df['valid'].notna()).sum()}")

valid: 12294
out of bounds: 701


In [ ]:
#merge geocoded_locations with main df
df = df.drop(columns=['imp_lat', 'imp_lng', 'final_lat', 'final_lng'], errors='ignore')

df = df.merge(geocoded_df[['location', 'imp_lat', 'imp_lng']], on='location', how='left')

# use original coords where available, fall back to imputed
df['final_lat'] = df['lat'].fillna(df['imp_lat'])
df['final_lng'] = df['lng'].fillna(df['imp_lng'])

print(f"original coords: {df['lat'].notna().sum()}")
print(f"imputed coords: {(df['lat'].isna() & df['final_lat'].notna()).sum()}")
print(f"still missing: {df['final_lat'].isna().sum()}")
print(f"coverage: {df['final_lat'].notna().mean():.1%}")

original coords: 260408
imputed coords: 150517
still missing: 101167
coverage: 80.2%


In [57]:
print(f"sample of locations missed {missing[missing['location'].notna()]['location'].sample(20).tolist()}")
print(f"no coords + no location string: {missing['location'].isna().sum()}")
print(f"no coords but have location string: {missing['location'].notna().sum()}")

sample of locations missed ['123XX N I-10 Service Rd', '015XX Rev John Raphael Jr Way', "027XX D'Abadie St", '004XX No Hammond Hwy', '123XX N I-10 Service Rd', '031XX De Saix Blvd', '014XX Slidell St', '120XX S I-10 Service Rd', '004XX S Jefferson Davis Pkwy', '060XX Kuebel Dr', '100XX N I-10 Service Rd', '004XX Slidell St', '011XX Odeon Ave', '017XX 4th St', '121XX N I-10 Service Rd', '048XX Bergerac St', '075XX Burg', '038XX Mac Arthur Blvd', '042XX Aster St', '021XX 1st St']
no coords + no location string: 95986
no coords but have location string: 5181


In [5]:
# clean up merged df
df = df.drop(columns=['imp_lat_x', 'imp_lng_x', 'imp_lat_y', 'imp_lng_y', 'imp_lat', 'imp_lng'], errors='ignore')
df.sample(10)
df.to_csv("policing_imputed_coords.csv")